# Data Registry for Satellite Data from Planet Labs

_by Alex Berndt_
_9 Dec 2021_

__Abstract__

This notebook serves as an entrypoint into what a Spatio-Temporal Asset Catalog (STAC) is, and how we use this at Overstory.

__References__

1. [Example STAC Catalog from Planet Labs](https://developers.planet.com/planetschool/introduction-to-stac-part-2-creating-an-example-stac-catalog-of-planet-imagery-with-pystac/)

### Import Dependencies

In [1]:
import os
import json
import requests
import urllib.request
import rasterio
from tempfile import TemporaryDirectory
from pathlib import Path
import typing
import pystac
from pystac import (Catalog, CatalogType)
from typing import Dict, List, Tuple
from logging import DEBUG, Logger
from rasterio.warp import calculate_default_transform
from shapely.geometry import Polygon, mapping
from datetime import datetime
from itertools import islice

from helper import (
    read_json, 
    get_metadata_path,
    get_udm_file_path,
    get_udm2_file_path,
    get_tif_file_path,
    get_COG_file_path,
)

# Define core repository directories
ROOT_DIR = Path(os.getcwd()).parents[2]
ASSETS_DIR = os.path.join(ROOT_DIR, "assets/")
CATALOG_DIR = os.path.join(ROOT_DIR, "catalog/")

## 1. Ingest Planet Data into STAC Registry

### Read `*_metadata.json` Files

In [2]:
# Define item id's saved in `./assets/` dir
item_ids = [
    '20210425_100412_ssc3_u0001',
    '20210425_100412_ssc3_u0002',
    '20210528_131522_ssc9_u0001',
    '20210710_133141_ssc8_u0001',
    '20210922_100732_ssc12_u0001',
    '20210922_100732_ssc12_u0002',
]

# Loop through each item
for item_id in item_ids:
    
    metadata_file = get_metadata_path(item_id)
    udm_file = get_udm_file_path(item_id)
    udm2_file = get_udm2_file_path(item_id)
    tif_file = get_tif_file_path(item_id)
    cog_file = get_COG_file_path(item_id)
    
    metadata_data = dict(read_json(metadata_file))
    
    print("----------------------------")
    print(metadata_data['id'])
    print(metadata_data['properties'])
    print(metadata_data['properties']["acquired"][:-1])
    print(f'udm_file:  {udm_file}')
    print(f'udm2_file: {udm2_file}')
    print(f'tif_file:  {tif_file}')
    print(f'cog_file:  {cog_file}')

----------------------------
20210425_100412_ssc3_u0001
{'acquired': '2021-04-25T10:04:12.75Z', 'clear_confidence_percent': 91, 'clear_percent': 93, 'cloud_cover': 0, 'cloud_percent': 0, 'ground_control_ratio': 1, 'gsd': 0.81, 'heavy_haze_percent': 0, 'item_type': 'SkySatCollect', 'light_haze_percent': 0, 'pixel_resolution': 0.5, 'provider': 'skysat', 'published': '2021-04-25T14:18:33Z', 'publishing_stage': 'finalized', 'quality_category': 'standard', 'satellite_azimuth': 35.7, 'satellite_id': 'SSC3', 'shadow_percent': 3, 'snow_ice_percent': 0, 'strip_id': 's103_20210425T100412Z', 'sun_azimuth': 145.6, 'sun_elevation': 51.7, 'updated': '2021-09-11T06:44:43Z', 'view_angle': 28.3, 'visible_confidence_percent': 78, 'visible_percent': 96}
2021-04-25T10:04:12.75
udm_file:  /home/alex/github_repos/ml-notebooks/assets/20210425_100412_ssc3_u0001/files/SkySatCollect/20210425_100412_ssc3_u0001/pansharpened_udm2/20210425_100412_ssc3_u0001_pansharpened_udm.tif
udm2_file: /home/alex/github_repos/ml

### Create STAC Items

In [3]:
# Import python packages
from rasterio.warp import calculate_default_transform
from shapely.geometry import Polygon, mapping
from datetime import datetime
from itertools import islice

In [4]:
def create_STAC_Item(
    tiff_path: str, 
    metadata_json: Dict,
    catalog: pystac.STACObject,
) -> Tuple[pystac.Item, Tuple]:
    """Creates a STAC item.
    
    Args:
        tiff_path (str): path to the COG file.
        metadata_json (Dict): dict content of metadata.json file.
        
    Returns:
        List of STAC items.
    """

    with rasterio.open(tiff_path) as sample_cog:
        
        bounds = sample_cog.bounds
        src_crs = sample_cog.crs
        dst_crs = 'EPSG:4326'  # EPSG identifier for WGS84 coordinate system used by the geojson format
        
        left, bottom, right, top = rasterio.warp.transform_bounds(sample_cog.crs, dst_crs, *bounds)
        bbox = [left, bottom, right, top]
        
        # Create geojson feature
        geom = mapping(Polygon([
          [left, bottom],
           [left, top],
           [right, top],
           [right, bottom]
        ]))
        
        time_acquired = datetime.strptime(metadata_json["properties"]["acquired"][:-1], '%Y-%m-%dT%H:%M:%S.%f')
        
        # Instantiate pystac item
        item = pystac.Item(id=metadata_json["id"],
                             geometry=geom,
                             bbox=bbox,
                             datetime=time_acquired,
                             properties={},
                          )
        
        # links=[pystac.Link(rel='item', target=catalog)]

        # Use Planet metadata.json to add some common metadata to the STAC item
        metadata_properties = metadata_json["properties"]
        
        # TODO: Enable item extensions
        # item.ext.enable('eo')
        # item.ext.enable('view') 
        # item.ext.enable('projection')
        
        for key, value in islice(metadata_properties.items(), 1, None):

            # Add some common metadata for the item not included in the core item specification
            
            # Ground Sample Distance
            if(key == 'gsd'):
                item.common_metadata.gsd = value
                
            # TODO: Could not add item.ext - item has no ext? Need to investigate

        # Tuple containing spatial and temporal extent information to use later in this tutorial
        item_extent_info =  (bbox, geom, time_acquired)
     
    # Returns a list containing the PySTAC Item object and a tuple 
    # holding the bounding box, geojson polygon, and date the item was acquired
    return item, (item_extent_info)

def create_STAC_Items(item_ids: List[str], catalog: pystac.STACObject) -> List[Tuple]:
    """ Create STAC Items.
    
    Args:
        item_ids (List[str]):  
    
    Returns:
        stac_items (List[Tuple]): list of STAC items
    """
    item_ids = sorted(item_ids, reverse=True)

    # empty list to store STAC items
    stac_items = []
    
    for item_id in item_ids:
        
        metadata_path = get_metadata_path(item_id)
        metadata_json = read_json(metadata_path)
        url_udm = get_udm_file_path(item_id)
        url_udm2 = get_udm2_file_path(item_id)
        url_tif = get_tif_file_path(item_id)
        url_cog = get_COG_file_path(item_id)

        # Create STAC Items
        item, extent = create_STAC_Item(
            tiff_path=url_tif, 
            metadata_json=metadata_json,
            catalog=catalog,
        )

        # Add COG asset to STAC items list
        item.add_asset(
              key='COG',
              asset=pystac.Asset(
                  href=url_cog,
                  title= f"{item_id} - COG PSScene4Band",
                  # indicate it is a cloud optimized geotiff
                  media_type=pystac.MediaType.COG,
                  roles=([
                    "analytic"
                  ])
              )
        ) 
        
        # Add UDM2 asset to STAC items list
        item.add_asset(
              key='UDM2',
              asset=pystac.Asset(
                  href=url_udm2,
                  title= f"{item_id} - Usable Data Masks (UDM2)",
                  media_type=pystac.MediaType.TIFF,
                  roles=([
                    "analytic"
                  ])
              )
        ) 
        
        # Add UDM asset to STAC items list
        item.add_asset(
              key='UDM',
              asset=pystac.Asset(
                  href=url_udm,
                  title= f"{item_id} - Unusable Data Masks (UDM)",
                  media_type=pystac.MediaType.TIFF,
                  roles=([
                    "analytic"
                  ])
              )
        )
        
        # Add catalog to be parent link
        item.add_link(pystac.Link(rel=pystac.RelType.PARENT, target=catalog))
        
        stac_items.append((item, extent))
    return stac_items

### Add Items to STAC Catalog

In [5]:
catalog = pystac.Catalog(id='overstory-stac', description='Overstory STAC Catalog.')

# collection = pystac.Collection(
#     id='overstory-collection', 
#     description='Collection of provided satellite imagery.',
#     extent=pystac.Extext(
#         spatial=pystac.collection.SpatialExtent(
#             bboxes=))
# )

stac_items = create_STAC_Items(item_ids=item_ids, catalog=catalog)

for _, item in enumerate(stac_items):
    catalog.add_item(item[0])
    # print(json.dumps(item[0].to_dict(), indent=4))
    
catalog.describe()

* <Catalog id=overstory-stac>
  * <Item id=20210922_100732_ssc12_u0002>
  * <Item id=20210922_100732_ssc12_u0001>
  * <Item id=20210710_133141_ssc8_u0001>
  * <Item id=20210528_131522_ssc9_u0001>
  * <Item id=20210425_100412_ssc3_u0002>
  * <Item id=20210425_100412_ssc3_u0001>


### Add Links for Search

In [6]:
for item in stac_items:
    catalog.add_link(pystac.Link(rel="search", target=item[0]))
    
    print(type(item[0]))

<class 'pystac.item.Item'>
<class 'pystac.item.Item'>
<class 'pystac.item.Item'>
<class 'pystac.item.Item'>
<class 'pystac.item.Item'>
<class 'pystac.item.Item'>


### STAC Validation

Validate contents of the STAC recursively to ensure all objects within it follow the latest set of JSON schemas (see [here](https://pystac.readthedocs.io/en/1.0/concepts.html#validating-pystac-objects)).

In [7]:
catalog.normalize_hrefs(CATALOG_DIR)
catalog.validate_all()

### Save Catalog Locally

Save the catalog locally to file.

In [8]:
catalog.save(catalog_type=CatalogType.SELF_CONTAINED)

In [9]:
with open(catalog.get_self_href()) as f:
    print(f.read())

{
  "type": "Catalog",
  "id": "overstory-stac",
  "stac_version": "1.0.0",
  "description": "Overstory STAC Catalog.",
  "links": [
    {
      "rel": "root",
      "href": "./catalog.json",
      "type": "application/json"
    },
    {
      "rel": "item",
      "href": "./20210922_100732_ssc12_u0002/20210922_100732_ssc12_u0002.json",
      "type": "application/json"
    },
    {
      "rel": "item",
      "href": "./20210922_100732_ssc12_u0001/20210922_100732_ssc12_u0001.json",
      "type": "application/json"
    },
    {
      "rel": "item",
      "href": "./20210710_133141_ssc8_u0001/20210710_133141_ssc8_u0001.json",
      "type": "application/json"
    },
    {
      "rel": "item",
      "href": "./20210528_131522_ssc9_u0001/20210528_131522_ssc9_u0001.json",
      "type": "application/json"
    },
    {
      "rel": "item",
      "href": "./20210425_100412_ssc3_u0002/20210425_100412_ssc3_u0002.json",
      "type": "application/json"
    },
    {
      "rel": "item",
      "href"

## 2. Querying the STAC Catalog

The objective here is to query our newly created STAC Catalog which we created in Part 1.

In [10]:
from pystac_client import Client

catalog_client = Client.open(os.path.join(CATALOG_DIR, 'catalog.json'))

In [11]:
catalog_client

<Client id=overstory-stac>

In [12]:
items = catalog.get_items()
list(items)

[<Item id=20210922_100732_ssc12_u0002>,
 <Item id=20210922_100732_ssc12_u0001>,
 <Item id=20210710_133141_ssc8_u0001>,
 <Item id=20210528_131522_ssc9_u0001>,
 <Item id=20210425_100412_ssc3_u0002>,
 <Item id=20210425_100412_ssc3_u0001>]

In [149]:
# landsat = catalog.get_child("landsat-8-c2-l2")
# for band in landsat.extra_fields["summaries"]["eo:bands"]:
#     name = band["name"]
#     description = band["description"]
#     common_name = "" if "common_name" not in band else f"({band['common_name']})"
#     ground_sample_distance = band["gsd"]
#     print(f"{name} {common_name}: {description} ({ground_sample_distance}m resolution)")

In [150]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [7.2, 47.0],
            [7.5, 47.0],
            [7.5, 47.8],
            [7.2, 47.8],
            [7.2, 47.0],
        ]
    ],
}
time_range = "2021-04-20/2021-09-26"

search = catalog_client.search(
    intersects=area_of_interest, datetime=time_range
)

In [151]:
search

In [153]:
items = list(search.get_items())
for item in items:
    print(f"{item.id}: {item.datetime}")

### STAC Query backup

There is a strange 

In [13]:
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-122.27508544921875, 47.54687159892238],
            [-121.96128845214844, 47.54687159892238],
            [-121.96128845214844, 47.745787772920934],
            [-122.27508544921875, 47.745787772920934],
            [-122.27508544921875, 47.54687159892238],
        ]
    ],
}

time_range = "2020-12-01/2020-12-31"

search = catalog.search(
    collections=["landsat-8-c2-l2"], intersects=area_of_interest, datetime=time_range
)

In [14]:
items = list(search.get_items())
for item in items:
    print(f"{item.id}: {item.datetime}")

LC08_L2SP_046027_20201229_02_T2: 2020-12-29 18:55:56.738265+00:00
LC08_L2SP_047027_20201220_02_T2: 2020-12-20 19:02:09.878796+00:00
LC08_L2SP_046027_20201213_02_T2: 2020-12-13 18:56:00.096447+00:00
LC08_L2SP_047027_20201204_02_T1: 2020-12-04 19:02:11.194486+00:00
